In [174]:
from typing import Dict, List, Tuple, Set
import regex
import json

def bpe_train(dataset: str, num_merges: int) -> Dict[int, str]:
    pass

def read_corpus(filename: str) -> Dict[int, str]:
    with open(filename, "r", encoding="utf-8") as f:
        data = f.read().encode("utf-8")
        print("Len(data)", len(data))
        print(data[1000:1100])

read_corpus("data/taylor_swift_wiki.txt")

encoding_map: Dict[str, int] = {}

with open("weights/gpt2_encoder.json", "rb") as f:
    for k, v in json.loads(f.read()).items():
        encoding_map[k] = v

with open("weights/gpt2_vocab.bpe", "rb") as f:
    merges: Dict[Tuple[bytes, bytes], int] = {tuple(l[:-1].split(b' ')): i+256 for i, l in enumerate(f.readlines()[1:])}

Len(data) 178347
b'\n\nTaylor Alison Swift (born December 13, 1989) is an American singer-songwriter. Known for her autob'


In [175]:
def get_stats(ints_list: List[int]):
    occurrences: Dict[Tuple[int, int], int] = {}
    for i in range(len(ints_list) - 1):
        pair = (ints_list[i], ints_list[i + 1])
        occurrences[pair] = occurrences.get(pair, 0) + 1
    return occurrences
    
def convert_merges_to_rtp(merges_map: Dict[Tuple[bytes, bytes], int]) -> Dict[int, bytes]:
    rank_to_pair: Dict[int, bytes] = {}
    for i in range(256):
        rank_to_pair[i] = bytes([i])
    for pair, rank in merges_map.items():
        rank_to_pair[rank] = pair[0] + pair[1]
    return rank_to_pair
        

def encode(merges_map: Dict[Tuple[bytes, bytes], int], rtp_map: Dict[int, bytes], str_to_encode: str) -> List[int]:
    for sub_string in str_to_encode.split(" "):
        current_str: List[int] = [int(i) for i in str_to_encode.encode("utf-8")]
        print(f"Current str: {current_str}")
        print(f"Original length: {len(current_str)}")
        not_found_strs: Set[Tuple[int, int]]= set()
        while True:
            stats = get_stats(current_str)
            print(stats.keys())
            large_number = 999999999
            merge_options = [(merges_map.get((rtp_map[pair[0]], rtp_map[pair[1]]), large_number), pair) 
                             for pair in stats.keys() 
                             if pair not in not_found_strs]
            if len(merge_options) == 0:
                print(f"New length {len(current_str)}")
                return current_str
            encoding_num, pair = min(merge_options)
            print(f"The min is {encoding_num} {pair}, merging: {rtp_map[pair[0]]} with {rtp_map[pair[1]]}")
            if encoding_num == large_number:
                not_found_strs.add(pair)
                continue
            new_str: List[int] = []
            i = 0
            while i < len(current_str):
                if (i + 1) < len(current_str) and (current_str[i], current_str[i+1]) == pair:
                    new_str.append(encoding_num)
                    print(f"Adding: {encoding_num}")
                    i += 2
                else:
                    new_str.append(current_str[i])
                    print(f"Adding: {current_str[i]}")
                    i += 1
            if len(current_str) == len(new_str):
                not_found_strs.add(pair)
            current_str = new_str

def decode(merges_map: Dict[Tuple[int,int], int], encoded: List[int]) -> str:
    
    pass

rtp = convert_merges_to_rtp(merges)
hello_str =  "hello world"
print(encode(merges, rtp, hello_str))
# list(merges.items())[:100]
list(rtp.items())[0:1000]

Current str: [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]
Original length: 11
dict_keys([(104, 101), (101, 108), (108, 108), (108, 111), (111, 32), (32, 119), (119, 111), (111, 114), (114, 108), (108, 100)])
The min is 258 (104, 101), merging: b'h' with b'e'
Adding: 258
Adding: 108
Adding: 108
Adding: 111
Adding: 32
Adding: 119
Adding: 111
Adding: 114
Adding: 108
Adding: 100
dict_keys([(258, 108), (108, 108), (108, 111), (111, 32), (32, 119), (119, 111), (111, 114), (114, 108), (108, 100)])
The min is 273 (111, 114), merging: b'o' with b'r'
Adding: 258
Adding: 108
Adding: 108
Adding: 111
Adding: 32
Adding: 119
Adding: 273
Adding: 108
Adding: 100
dict_keys([(258, 108), (108, 108), (108, 111), (111, 32), (32, 119), (119, 273), (273, 108), (108, 100)])
The min is 297 (108, 108), merging: b'l' with b'l'
Adding: 258
Adding: 297
Adding: 111
Adding: 32
Adding: 119
Adding: 273
Adding: 108
Adding: 100
dict_keys([(258, 297), (297, 111), (111, 32), (32, 119), (119, 273), (273, 108), (10

[(0, b'\x00'),
 (1, b'\x01'),
 (2, b'\x02'),
 (3, b'\x03'),
 (4, b'\x04'),
 (5, b'\x05'),
 (6, b'\x06'),
 (7, b'\x07'),
 (8, b'\x08'),
 (9, b'\t'),
 (10, b'\n'),
 (11, b'\x0b'),
 (12, b'\x0c'),
 (13, b'\r'),
 (14, b'\x0e'),
 (15, b'\x0f'),
 (16, b'\x10'),
 (17, b'\x11'),
 (18, b'\x12'),
 (19, b'\x13'),
 (20, b'\x14'),
 (21, b'\x15'),
 (22, b'\x16'),
 (23, b'\x17'),
 (24, b'\x18'),
 (25, b'\x19'),
 (26, b'\x1a'),
 (27, b'\x1b'),
 (28, b'\x1c'),
 (29, b'\x1d'),
 (30, b'\x1e'),
 (31, b'\x1f'),
 (32, b' '),
 (33, b'!'),
 (34, b'"'),
 (35, b'#'),
 (36, b'$'),
 (37, b'%'),
 (38, b'&'),
 (39, b"'"),
 (40, b'('),
 (41, b')'),
 (42, b'*'),
 (43, b'+'),
 (44, b','),
 (45, b'-'),
 (46, b'.'),
 (47, b'/'),
 (48, b'0'),
 (49, b'1'),
 (50, b'2'),
 (51, b'3'),
 (52, b'4'),
 (53, b'5'),
 (54, b'6'),
 (55, b'7'),
 (56, b'8'),
 (57, b'9'),
 (58, b':'),
 (59, b';'),
 (60, b'<'),
 (61, b'='),
 (62, b'>'),
 (63, b'?'),
 (64, b'@'),
 (65, b'A'),
 (66, b'B'),
 (67, b'C'),
 (68, b'D'),
 (69, b'E'),
 (70, b'F'

In [180]:
pat_str = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
gpt2_regex = regex.compile(pat_str)
gpt2_regex.findall("This is me saying helllo to the world.\n I don't know if the world wants it though... 125 + 524 = 12321, 1,2,3, 14,15,65!")


['This',
 ' is',
 ' me',
 ' saying',
 ' helllo',
 ' to',
 ' the',
 ' world',
 '.',
 '\n',
 ' I',
 ' don',
 "'t",
 ' know',
 ' if',
 ' the',
 ' world',
 ' wants',
 ' it',
 ' though',
 '...',
 ' 125',
 ' +',
 ' 524',
 ' =',
 ' 12321',
 ',',
 ' 1',
 ',',
 '2',
 ',',
 '3',
 ',',
 ' 14',
 ',',
 '15',
 ',',
 '65',
 '!']